### Import Package

In [1]:
import os
print(os.path.exists(".."))

True


In [2]:
import sys

sys.path.append("..")

In [3]:
from app.model.products_model import Product, Specification
from app.database.session import get_session, create_db

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
import pandas as pd
import ast
from typing import Optional

In [6]:
df = pd.read_csv("../dataset/product.csv")

In [7]:
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,formatted_specifications,product_category
0,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,aw bellies,"['footwear', 'womens footwear', 'ballerinas', ...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,key features of aw bellies sandals wedges heel...,0.0,0.0,aw,"[{'key': 'Ideal For', 'value': 'Women'}, {'key...","ideal for: women, occasion: casual, color: red...","footwear, womens footwear, ballerinas, aw bellies"
1,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,sicons all purpose arnica dog shampoo,"['pet supplies', 'grooming', 'skin coat care'...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,specifications of sicons all purpose arnica do...,0.0,0.0,sicons,"[{'key': 'Pet Type', 'value': 'Dog'}, {'key': ...","pet type: dog, brand: sicons, quantity: 500 ml...","pet supplies, grooming, skin coat care, shamp..."
2,c2a17313954882c1dba461863e98adf2,2016-03-25 22:59:23 +0000,http://www.flipkart.com/eternal-gandhi-super-s...,eternal gandhi super series crystal paper weig...,['eternal gandhi super series crystal paper we...,PWTEB7H2E4KCYUE3,430.0,430.0,"[""http://img5a.flixcart.com/image/paper-weight...",False,key features of eternal gandhi super series cr...,0.0,0.0,eternal gandhi,"[{'key': 'Model Name', 'value': 'Gandhi Paper ...","model name: gandhi paper weight mark v, weight...",eternal gandhi super series crystal paper weight
3,29c8d290caa451f97b1c32df64477a2c,2016-03-25 22:59:23 +0000,http://www.flipkart.com/dilli-bazaaar-bellies-...,"dilli bazaaar bellies, corporate casuals, casuals","['footwear', 'womens footwear', 'ballerinas', ...",SHOEH3DZBFR88SCK,699.0,349.0,"[""http://img6a.flixcart.com/image/shoe/b/p/n/p...",False,"key features of dilli bazaaar bellies, corpora...",0.0,0.0,dilli bazaaar,"[{'key': 'Occasion', 'value': 'Ethnic, Casual,...","occasion: ethnic, casual, party, formal, ideal...","footwear, womens footwear, ballerinas, dilli b..."
4,e54bc0a7c3429da2ebef0b30331fe3d2,2016-03-25 22:59:23 +0000,http://www.flipkart.com/ladela-bellies/p/itmeh...,ladela bellies,"['footwear', 'womens footwear', 'ballerinas', ...",SHOEH4KM2W3Z6EH5,1724.0,950.0,"[""http://img5a.flixcart.com/image/shoe/s/g/m/b...",False,key features of ladela bellies brand: ladela c...,5.0,5.0,ladela,"[{'key': 'Occasion', 'value': 'Casual'}, {'key...","occasion: casual, ideal for: women, heel heigh...","footwear, womens footwear, ballerinas, ladela ..."


In [8]:
len(df)

12676

### Fill NA

In [9]:
df["product_name"] = df["product_name"].fillna("-")
df["description"] = df["description"].fillna("-")
df["brand"] = df["brand"].fillna("-")
df["formatted_specifications"] = df["formatted_specifications"].fillna("-")
df["image"] = df["image"].fillna("-")

In [10]:
df.isna().sum()

uniq_id                     0
crawl_timestamp             0
product_url                 0
product_name                0
product_category_tree       0
pid                         0
retail_price                0
discounted_price            0
image                       0
is_FK_Advantage_product     0
description                 0
product_rating              0
overall_rating              0
brand                       0
product_specifications      0
formatted_specifications    0
product_category            0
dtype: int64

### Prepare Data For Postgres

In [11]:
df.head(1)

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,formatted_specifications,product_category
0,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,aw bellies,"['footwear', 'womens footwear', 'ballerinas', ...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,key features of aw bellies sandals wedges heel...,0.0,0.0,aw,"[{'key': 'Ideal For', 'value': 'Women'}, {'key...","ideal for: women, occasion: casual, color: red...","footwear, womens footwear, ballerinas, aw bellies"


In [12]:
df["product_specifications"].head(10).tolist()

["[{'key': 'Ideal For', 'value': 'Women'}, {'key': 'Occasion', 'value': 'Casual'}, {'key': 'Color', 'value': 'Red'}, {'key': 'Outer Material', 'value': 'Patent Leather'}, {'key': 'Heel Height', 'value': '1 inch'}, {'key': 'Number of Contents in Sales Package', 'value': 'Pack of 1'}, {'value': 'One Pair Of Shoes'}]",
 "[{'key': 'Pet Type', 'value': 'Dog'}, {'key': 'Brand', 'value': 'Sicons'}, {'key': 'Quantity', 'value': '500 ml'}, {'key': 'Model Number', 'value': 'SH.DF-14'}, {'key': 'Type', 'value': 'All Purpose'}, {'key': 'Fragrance', 'value': 'Arnica'}, {'key': 'Form Factor', 'value': 'Liquid'}, {'key': 'Sales Package', 'value': 'Shampoo Sicons Dog Fashion Arnica'}]",
 "[{'key': 'Model Name', 'value': 'Gandhi Paper Weight Mark V'}, {'key': 'Weight', 'value': '323 g'}, {'value': 'Paper Weight'}, {'key': 'Paper Weight Material', 'value': 'Crystal'}, {'key': 'Paper Weight Finish', 'value': 'Silver Finish'}]",
 "[{'key': 'Occasion', 'value': 'Ethnic, Casual, Party, Formal'}, {'key': 'Id

In [14]:
products = []
specifications = []


def parse_spec_list(raw_spec: str) -> Optional[list]:
    try:
        parsed = ast.literal_eval(raw_spec)
        if isinstance(parsed, list):
            return parsed
    except Exception as e:
        print(f"Spec parsing error: {e}")
    return None


for _, row in df.iterrows():
    # Create Product instance
    prod = Product(
        id=row["uniq_id"],
        crawl_timestamp=str(row["crawl_timestamp"]),
        product_url=row["product_url"],
        product_name=row["product_name"],
        product_category_tree=str(row["product_category_tree"]),
        pid=row["pid"],
        retail_price=float(row["retail_price"]),
        discounted_price=float(row["discounted_price"]),
        image=str(row["image"]),
        is_fk_advantage_product=bool(row["is_FK_Advantage_product"]),
        description=row.get("description"),
        product_rating=float(row["product_rating"]),
        overall_rating=float(row["overall_rating"]),
        brand=row["brand"],
        formatted_specifications=row["formatted_specifications"],
        product_category=row["product_category"],
    )
    products.append(prod)

    # Parse and extract valid specifications
    raw_specs = row.get("product_specifications")
    if raw_specs:
        parsed_specs = parse_spec_list(raw_specs)
        if parsed_specs:
            for item in parsed_specs:
                if isinstance(item, dict) and "key" in item and "value" in item:
                    specifications.append(
                        Specification(
                            key=item["key"],
                            value=item["value"],
                            product_id=row["uniq_id"],
                        )
                    )

In [15]:
products[0]

Product(id='f449ec65dcbc041b6ae5e6a32717d01b', crawl_timestamp='2016-03-25 22:59:23 +0000', product_url='http://www.flipkart.com/aw-bellies/p/itmeh4grgfbkexnt?pid=SHOEH4GRSUBJGZXE', product_name='aw bellies', product_category_tree="['footwear', 'womens footwear', 'ballerinas', 'aw bellies']", pid='SHOEH4GRSUBJGZXE', retail_price=999.0, discounted_price=499.0, image='["http://img5a.flixcart.com/image/shoe/7/z/z/red-as-454-aw-11-original-imaeebfwsdf6jdf6.jpeg", "http://img6a.flixcart.com/image/shoe/7/z/z/red-as-454-aw-11-original-imaeebfwsdf6jdf6.jpeg", "http://img5a.flixcart.com/image/shoe/7/z/z/red-as-454-aw-11-original-imaeebfwt8zqc5cy.jpeg", "http://img5a.flixcart.com/image/shoe/7/z/z/red-as-454-aw-11-original-imaeebfwhz5f8v6m.jpeg", "http://img6a.flixcart.com/image/shoe/7/z/z/red-as-454-aw-11-original-imaeeducshzc6pbz.jpeg", "http://img6a.flixcart.com/image/shoe/7/z/z/red-as-454-aw-11-original-imaeebfw2veeuz8s.jpeg"]', is_fk_advantage_product=False, description='key features of aw b

In [16]:
specifications[0]

Specification(key='Ideal For', value='Women', product_id='f449ec65dcbc041b6ae5e6a32717d01b', id=None)

### Bulk Inset Into Postgres

In [19]:
create_db()

2025-04-19 02:00:06,398 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-19 02:00:06,419 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-04-19 02:00:06,421 INFO sqlalchemy.engine.Engine [generated in 0.00211s] {'table_name': 'product', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-04-19 02:00:06,435 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_c

In [20]:
with next(get_session()) as session:
    session.add_all(products)
    session.commit()

2025-04-19 02:00:11,515 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-19 02:00:11,852 INFO sqlalchemy.engine.Engine INSERT INTO product (id, crawl_timestamp, product_url, product_name, product_category_tree, pid, retail_price, discounted_price, image, is_fk_advantage_product, description, product_rating, overall_rating, brand, formatted_specifications, product_cat ... 380152 characters truncated ... overall_rating__999)s, %(brand__999)s, %(formatted_specifications__999)s, %(product_category__999)s)
2025-04-19 02:00:11,854 INFO sqlalchemy.engine.Engine [cached since 47.89s ago (insertmanyvalues) 1/13 (unordered)] {'image__0': '["http://img5a.flixcart.com/image/shoe/7/z/z/red-as-454-aw-11-original-imaeebfwsdf6jdf6.jpeg", "http://img6a.flixcart.com/image/shoe/7/z/z/red-as-454 ... (266 characters truncated) ... 7/z/z/red-as-454-aw-11-original-imaeeducshzc6pbz.jpeg", "http://img6a.flixcart.com/image/shoe/7/z/z/red-as-454-aw-11-original-imaeebfw2veeuz8s.jpeg"]', 'product_rating__0':

In [21]:
with next(get_session()) as session:
    session.add_all(specifications)
    session.commit()

2025-04-19 02:00:23,440 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-19 02:00:25,353 INFO sqlalchemy.engine.Engine INSERT INTO specification (key, value, product_id) SELECT p0::VARCHAR, p1::VARCHAR, p2::VARCHAR FROM (VALUES (%(key__0)s, %(value__0)s, %(product_id__0)s, 0), (%(key__1)s, %(value__1)s, %(product_id__1)s, 1), (%(key__2)s, %(value__2)s, %(product_id__ ... 57429 characters truncated ... (p0, p1, p2, sen_counter) ORDER BY sen_counter RETURNING specification.id, specification.id AS id__1
2025-04-19 02:00:25,354 INFO sqlalchemy.engine.Engine [generated in 0.48458s (insertmanyvalues) 1/145 (ordered)] {'value__0': 'Women', 'key__0': 'Ideal For', 'product_id__0': 'f449ec65dcbc041b6ae5e6a32717d01b', 'value__1': 'Casual', 'key__1': 'Occasion', 'product_id__1': 'f449ec65dcbc041b6ae5e6a32717d01b', 'value__2': 'Red', 'key__2': 'Color', 'product_id__2': 'f449ec65dcbc041b6ae5e6a32717d01b', 'value__3': 'Patent Leather', 'key__3': 'Outer Material', 'product_id__3': 'f449ec65dcbc0